In [ ]:
#Taufik Kurahman Sasmita
#1103200008
#Modern_CNN_GoogleNet

In [1]:
pip install d2l

##**Library**

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from d2l import tensorflow as d2l

##**Kelas Inception Block**


In [19]:
import tensorflow as tf
from tensorflow.keras import layers

class InceptionBlock(tf.keras.layers.Layer):
    def __init__(self, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, **kwargs):
        super(InceptionBlock, self).__init__(**kwargs)

        # 1x1 convolution branch
        self.branch1x1 = layers.Conv2D(filters_1x1, kernel_size=1, activation='relu')

        # 3x3 convolution branch
        self.branch3x3 = tf.keras.Sequential([
            layers.Conv2D(filters_3x3_reduce, kernel_size=1, activation='relu'),
            layers.Conv2D(filters_3x3, kernel_size=3, padding='same', activation='relu')
        ])

        # 5x5 convolution branch
        self.branch5x5 = tf.keras.Sequential([
            layers.Conv2D(filters_5x5_reduce, kernel_size=1, activation='relu'),
            layers.Conv2D(filters_5x5, kernel_size=5, padding='same', activation='relu')
        ])

        # Max pooling branch
        self.branch_pool = tf.keras.Sequential([
            layers.MaxPool2D(pool_size=3, strides=1, padding='same'),
            layers.Conv2D(filters_pool_proj, kernel_size=1, activation='relu')
        ])

    def call(self, inputs):
        branch1x1 = self.branch1x1(inputs)
        branch3x3 = self.branch3x3(inputs)
        branch5x5 = self.branch5x5(inputs)
        branch_pool = self.branch_pool(inputs)

        # Concatenate branches along the channel axis
        return tf.concat([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)


##**Pemodelan GoogleNet*

In [20]:
import tensorflow as tf
from tensorflow.keras import layers

# Definisi kelas Inception Block
class InceptionBlock(tf.keras.layers.Layer):
    def __init__(self, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, **kwargs):
        super(InceptionBlock, self).__init__(**kwargs)

        # 1x1 convolution branch
        self.branch1x1 = layers.Conv2D(filters_1x1, kernel_size=1, activation='relu')

        # 3x3 convolution branch
        self.branch3x3 = tf.keras.Sequential([
            layers.Conv2D(filters_3x3_reduce, kernel_size=1, activation='relu'),
            layers.Conv2D(filters_3x3, kernel_size=3, padding='same', activation='relu')
        ])

        # 5x5 convolution branch
        self.branch5x5 = tf.keras.Sequential([
            layers.Conv2D(filters_5x5_reduce, kernel_size=1, activation='relu'),
            layers.Conv2D(filters_5x5, kernel_size=5, padding='same', activation='relu')
        ])

        # Max pooling branch
        self.branch_pool = tf.keras.Sequential([
            layers.MaxPool2D(pool_size=3, strides=1, padding='same'),
            layers.Conv2D(filters_pool_proj, kernel_size=1, activation='relu')
        ])

    def call(self, inputs):
        branch1x1 = self.branch1x1(inputs)
        branch3x3 = self.branch3x3(inputs)
        branch5x5 = self.branch5x5(inputs)
        branch_pool = self.branch_pool(inputs)

        # Concatenate branches along the channel axis
        return tf.concat([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)

# Definisi model GoogleNet
class GoogleNet(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(GoogleNet, self).__init__()

        self.conv1 = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu')
        self.maxpool1 = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')

        self.conv2 = layers.Conv2D(64, kernel_size=1, activation='relu')
        self.conv3 = layers.Conv2D(192, kernel_size=3, padding='same', activation='relu')
        self.maxpool2 = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')

        self.inception3a = InceptionBlock(64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(128, 128, 192, 32, 96, 64)
        self.maxpool3 = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')

        self.inception4a = InceptionBlock(192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(256, 160, 320, 32, 128, 128)
        self.maxpool4 = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')

        self.inception5a = InceptionBlock(256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(384, 192, 384, 48, 128, 128)

        self.avgpool = layers.GlobalAveragePooling2D()
        self.dropout = layers.Dropout(0.4)
        self.fc = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = self.dropout(x)

        return self.fc(x)

# Contoh penggunaan model GoogleNet
# Buat instance model
model = GoogleNet(num_classes=10)

# Compile model dengan optimizer, loss function, dan metrik
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Tampilkan ringkasan arsitektur model
model.build(input_shape=(None, 224, 224, 3))
model.summary()


Model: "google_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          multiple                  9472      
                                                                 
 max_pooling2d_8 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          multiple                  4160      
                                                                 
 conv2d_32 (Conv2D)          multiple                  110784    
                                                                 
 max_pooling2d_9 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 inception_block (Inception  multiple                  1